## Getting Reddit Comment Data

In [2]:
import praw
from praw.models import MoreComments
import tqdm
import pandas as pd
import statistics
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
reddit = praw.Reddit(
    client_id="p1dG7hgoowK4BSlUdar1WQ",
    client_secret="pEePtSnw7KMDZi6fCzkKaOth_pgKpQ",
    password="outdoortuesday",
    user_agent="Big Data by u/DISWillJayminMaya ",
    username="DISWillJayminMaya ",
    prawcore_timeout = 5
)

In [4]:
analyzer = SentimentIntensityAnalyzer()

def returnListofScores(listofcomments):
    templist=[]
    for com in listofcomments:
        temp = analyzer.polarity_scores(com)
        compScore = temp['compound']
        templist.append(compScore)
    return templist


In [5]:
tickerlist = ['GME', 'Gamestop', 'SPY', 'TWTTR', 'Twitter', 'TSLA', 'Tesla', 'AMD']
subreddit = reddit.subreddit('wallstreetbets')

scoreDict={key:list() for key in tickerlist}

hot = subreddit.hot(limit=25) # getting first 15 posts in the 'hot' section of the subreddit
sum = [0] * len(tickerlist) # our output array
counttotal = 0 # total number of comment read
submissions_counter = 0

rel_comments = [] # List of comments that are relevant to the ticker list items

In [6]:
# Loop for fetching the comments and the amount of time each ticker is mentioned
for submissions in hot:
    if not submissions.stickied:
        submissions_counter+=1
        if submissions_counter > 5:
            comments = submissions.comments
            for comment in comments:
                if isinstance(comment, MoreComments):
                    continue
                counttotal+=1
                for i, ticker in enumerate(tickerlist):
                    if ticker in comment.body:
                        rel_comments.append(comment.body)
                        sum[i]=sum[i]+1
                        temp = returnCompScore(comment.body)
                        scoreDict[ticker].append(temp)

NameError: name 'returnCompScore' is not defined

In [7]:
meanDict={key:list() for key in tickerlist}
spreadDict={key:list() for key in tickerlist}

for tick in tickerlist:
    if len(scoreDict[tick])>0:
        meanDict[tick]=statistics.mean(scoreDict[tick])
    if len (scoreDict[tick])>1:
        spreadDict[tick]=statistics.stdev(scoreDict[tick])

In [8]:
print(meanDict)
print(spreadDict)

{'GME': [], 'Gamestop': [], 'SPY': [], 'TWTTR': [], 'Twitter': [], 'TSLA': [], 'Tesla': [], 'AMD': []}
{'GME': [], 'Gamestop': [], 'SPY': [], 'TWTTR': [], 'Twitter': [], 'TSLA': [], 'Tesla': [], 'AMD': []}


In [9]:
output=pd.DataFrame(data={'Tick': tickerlist, 'Counts': sum})
print('Total comments read: ',counttotal)
print(output[output['Counts']>0])

Total comments read:  9
   Tick  Counts
5  TSLA       1


## Getting Stock Information

In [10]:
import requests, os, json, csv

size = 'compact' # 'full' for complete historical data, 'compact' for most recent 100
ticker = ['GME', 'SPY', 'TWTTR', 'TSLA', 'AMD'] # stock tickers to search for
datatype = 'csv' # 'json' for JSON output, 'csv' for CSV output

for stock in ticker:
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock}&outputsize={size}&datatype={datatype}&apikey=QC1C7LRPUTLC597Q'
    response = requests.get(url)
    #Save CSV to file
    with open(f'{stock}.csv', 'wb') as file:
        file.write(response.content)

## Charting Data

In [11]:
amd_data = pd.read_csv("AMD.csv")
amd_data

,timestamp,open,high,low,close,volume
0,2022-04-29,88.05,91.790,85.3800,85.52,82647701
1,2022-04-28,86.67,90.580,84.7800,89.64,91495449
2,2022-04-27,84.25,87.900,84.0200,84.91,83125054
3,2022-04-26,89.74,90.120,85.0800,85.16,87805574
4,2022-04-25,89.86,91.370,88.6100,90.69,93481042
...,...,...,...,...,...,...
95,2021-12-13,138.25,139.400,133.4150,133.80,42173963
96,2021-12-10,141.29,141.365,135.8200,138.55,42224275
97,2021-12-09,145.16,146.690,137.8000,138.10,53019926
98,2021-12-08,144.96,147.040,142.7000,145.24,40977478


get avg, spread, median of sentiment of comments in any given period
    compare this with the performance of the stock

model:
    based on last x days of reddit comments, what is the price going to be?
    take data from x days, put it all into one vector, and predict from this
    
    

## Making a function to return all comments that mention a stock based on a given date range

In [12]:
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta

def make_request(uri, max_retries = 5):
    """
    Function taken from medium article:
    https://medium.com/@pasdan/how-to-scrap-reddit-using-pushshift-io-via-python-a3ebcc9b83f4
    """
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)
    current_tries = 1
    while current_tries < max_retries:
        try:
            time.sleep(1)
            response = fire_away(uri)
            return response
        except:
            time.sleep(1)
            current_tries += 1
    return fire_away(uri)

def get_intervals(startDate, endDate, daysInInterval = 1):
    """
    get_intervals goes day by day through the start and end dates, returning that day's POSTIX
    """
    # Converting start and end dates to POSTIX:
    startPOSTIX = math.floor(startDate.timestamp())
    endPOSTIX = math.floor(endDate.timestamp())
    # 86,400 seconds in a day:
    period = (86400 * daysInInterval)
    end = startPOSTIX + period
    
    yield(int(startPOSTIX), int(end))
    
    padding = 1
    while end <= endPOSTIX:
        startPOSTIX = end + padding
        end = (startPOSTIX - padding) + period
        yield int(startPOSTIX), int(end)
    
    
def pull_posts_for(subreddit, start_at, end_at):
    """
    Function taken from medium article:
    https://medium.com/@pasdan/how-to-scrap-reddit-using-pushshift-io-via-python-a3ebcc9b83f4
    """
    def map_posts(posts):
        return list(map(lambda post: {
            'id': post['id'],
            'created_utc': post['created_utc'],
            'prefix': 't4_'
        }, posts))
    
    SIZE = 500
    URI_TEMPLATE = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&size={}'
    
    post_collections = map_posts(
        make_request(URI_TEMPLATE.format
                     (subreddit, start_at, end_at, SIZE))['data'])
    n = len(post_collections)
    while n == SIZE:
        last = post_collections[-1]
        new_start_at = last['created_utc'] - (10)
        
        more_posts = map_posts( \
            make_request( \
                URI_TEMPLATE.format( \
                    subreddit, new_start_at, end_at, bSIZE))['data'])
        
        n = len(more_posts)
        post_collections.extend(more_posts)
    return post_collections

def get_comments_by_date (startDate, endDate, subreddit='wallstreetbets'):
    """
    Takes a given time interval and scrapes the given subreddit for all of the comments
    that relate to the given ticker name, returning them as an array. Basic structure taken 
    from medium article.
    
    THIS CURRENTLY WILL NOT WORK IF GIVEN TODAY'S DATE. IT WILL ATTEMPT TO FETCH TOMORROW'S POSTS FOREVER
    """

    posts = []
    # This loop gets all of the posts in the given timeframe
    for interval in get_intervals(startDate, endDate):
        print("-- Fetching Posts From: ", datetime.fromtimestamp(interval[0]), " to ", datetime.fromtimestamp(interval[1]))
        pulled_posts = pull_posts_for(subreddit, interval[0], interval[1])
        posts.extend(pulled_posts)
#         time.sleep(.100) # So as not to over request reddit

    # Changing time stamps to be readable
    
    TIMEOUT_SECS = 1
    
    reddit_posts = []
    reddit_comments = {}
    startIndex = "{}-{}-{}".format(startDate.year, startDate.month, startDate.day)
    reddit_comments[startIndex] = []
    
    # Going through each unique post and comment and adding them to the relevant arrays
    #  WARNING: only looking at first 100 posts of each day
    for sub_id in np.unique([post['id'] for post in posts])[:100]:
        # Only looking at posts with more than 100 upvotes to speed the process up
        if reddit.submission(sub_id).ups > 100:
            sub = reddit.submission(id=sub_id)
            reddit_posts.append(sub)
            sub.comments.replace_more(limit=None)
            # Looping through each comment:
            temp_com_count = 0
            for comment in sub.comments.list()[:100]: 
                temp_com_count += 1
                reddit_comments[startIndex].append(comment.body)
                
            print("---- Fethced {} comments from post {}".format(temp_com_count, sub_id))
#             time.sleep(TIMEOUT_SECS)

    return reddit_comments

In [23]:
# Jan 11 - Jan 27 2021 For GME Boom
date_range = []

# Filling array with dates (should be 11, 28 for GME Boom)
# for day in range(11, 28):
#     date_range.append(datetime(2021, 1, day))
    
for day in range(10, 29):
    date_range.append(datetime(2022, 2, day))
    
for day in range(1, 32):
    date_range.append(datetime(2022, 3, day))
    
for day in range(1, 31):
    date_range.append(datetime(2022, 4, day))

In [24]:
date_range

[datetime.datetime(2022, 2, 10, 0, 0),
 datetime.datetime(2022, 2, 11, 0, 0),
 datetime.datetime(2022, 2, 12, 0, 0),
 datetime.datetime(2022, 2, 13, 0, 0),
 datetime.datetime(2022, 2, 14, 0, 0),
 datetime.datetime(2022, 2, 15, 0, 0),
 datetime.datetime(2022, 2, 16, 0, 0),
 datetime.datetime(2022, 2, 17, 0, 0),
 datetime.datetime(2022, 2, 18, 0, 0),
 datetime.datetime(2022, 2, 19, 0, 0),
 datetime.datetime(2022, 2, 20, 0, 0),
 datetime.datetime(2022, 2, 21, 0, 0),
 datetime.datetime(2022, 2, 22, 0, 0),
 datetime.datetime(2022, 2, 23, 0, 0),
 datetime.datetime(2022, 2, 24, 0, 0),
 datetime.datetime(2022, 2, 25, 0, 0),
 datetime.datetime(2022, 2, 26, 0, 0),
 datetime.datetime(2022, 2, 27, 0, 0),
 datetime.datetime(2022, 2, 28, 0, 0),
 datetime.datetime(2022, 3, 1, 0, 0),
 datetime.datetime(2022, 3, 2, 0, 0),
 datetime.datetime(2022, 3, 3, 0, 0),
 datetime.datetime(2022, 3, 4, 0, 0),
 datetime.datetime(2022, 3, 5, 0, 0),
 datetime.datetime(2022, 3, 6, 0, 0),
 datetime.datetime(2022, 3, 7, 

In [25]:

for day in date_range:
    end_day = day
    start_day = day - timedelta(days=1)
    temp_all_coms = get_comments_by_date(start_day, end_day)
    all_comments.update(temp_all_coms)

-- Fetching Posts From:  2022-02-09 00:00:00  to  2022-02-10 00:00:00
-- Fetching Posts From:  2022-02-10 00:00:01  to  2022-02-11 00:00:00
---- Fethced 100 comments from post sny145
---- Fethced 100 comments from post snydfx
---- Fethced 100 comments from post snzefu
---- Fethced 100 comments from post so19al
---- Fethced 100 comments from post so1djt
-- Fetching Posts From:  2022-02-10 00:00:00  to  2022-02-11 00:00:00
-- Fetching Posts From:  2022-02-11 00:00:01  to  2022-02-12 00:00:00
---- Fethced 100 comments from post soqt4d
---- Fethced 100 comments from post sose8q
---- Fethced 100 comments from post sotbru
---- Fethced 88 comments from post sou5lu
-- Fetching Posts From:  2022-02-11 00:00:00  to  2022-02-12 00:00:00
-- Fetching Posts From:  2022-02-12 00:00:01  to  2022-02-13 00:00:00
---- Fethced 100 comments from post spk92r
---- Fethced 34 comments from post spkisr
---- Fethced 54 comments from post spkmdr
---- Fethced 100 comments from post spmyd0
---- Fethced 42 comments

---- Fethced 82 comments from post t1jiga
---- Fethced 100 comments from post t1kpz6
---- Fethced 11 comments from post t1lth7
---- Fethced 100 comments from post t1lw4b
---- Fethced 57 comments from post t1lzrm
-- Fetching Posts From:  2022-02-27 00:00:00  to  2022-02-28 00:00:00
-- Fetching Posts From:  2022-02-28 00:00:01  to  2022-03-01 00:00:00
---- Fethced 100 comments from post t2b5ae
---- Fethced 100 comments from post t2bymv
---- Fethced 51 comments from post t2cfdx
---- Fethced 100 comments from post t2cmce
---- Fethced 55 comments from post t2cxyl
---- Fethced 100 comments from post t2dvem
---- Fethced 100 comments from post t2e5nu
---- Fethced 24 comments from post t2ejyd
---- Fethced 71 comments from post t2elzw
---- Fethced 100 comments from post t2foh2
---- Fethced 100 comments from post t2ft5t
---- Fethced 46 comments from post t2fx0v
---- Fethced 100 comments from post t2gwth
-- Fetching Posts From:  2022-02-28 00:00:00  to  2022-03-01 00:00:00
-- Fetching Posts From: 

---- Fethced 44 comments from post tdj9j3
---- Fethced 100 comments from post tdkbjy
---- Fethced 100 comments from post tdl15v
---- Fethced 100 comments from post tdl2nf
---- Fethced 99 comments from post tdlal7
---- Fethced 38 comments from post tdls6l
---- Fethced 93 comments from post tdmfc8
-- Fetching Posts From:  2022-03-15 00:00:00  to  2022-03-16 00:00:00
-- Fetching Posts From:  2022-03-16 00:00:01  to  2022-03-17 00:00:00
---- Fethced 65 comments from post tebdwk
---- Fethced 100 comments from post tecur0
---- Fethced 92 comments from post tee0ps
---- Fethced 70 comments from post teeg59
---- Fethced 69 comments from post teenwo
---- Fethced 100 comments from post tefvfg
-- Fetching Posts From:  2022-03-16 00:00:00  to  2022-03-17 00:00:00
-- Fetching Posts From:  2022-03-17 00:00:01  to  2022-03-18 00:00:00
---- Fethced 41 comments from post tf5l7q
---- Fethced 56 comments from post tf6fbq
---- Fethced 59 comments from post tf6tw2
---- Fethced 100 comments from post tf6vab


KeyboardInterrupt: 

In [ ]:
all_comments.keys()

In [26]:
tickerlist = ['GME', 'Gamestop', 'SPY', 'TWTTR', 'Twitter', 'TSLA', 'Tesla', 'AMD']
ticker_dict = {}

# Filling ticker_dict with empty dictionaries
for tick in tickerlist:
    ticker_dict[tick] = {}

    # Filling the dictionaries in ticker_dict with empty lists
for tick in tickerlist:
    for key in all_comments.keys():
        ticker_dict[tick][key] = []
        
# Adding the comments to their ticker and date
for tick in tickerlist:
    for key in all_comments.keys():
        for com in all_comments[key]:
            if tick in com:
                ticker_dict[tick][key].append(com)

In [28]:
all_comments.keys()

dict_keys(['2022-1-31', '2022-2-1', '2022-2-2', '2022-2-3', '2022-2-4', '2022-2-5', '2022-2-6', '2022-2-9', '2022-2-10', '2022-2-11', '2022-2-12', '2022-2-13', '2022-2-14', '2022-2-15', '2022-2-16', '2022-2-17', '2022-2-18', '2022-2-19', '2022-2-20', '2022-2-21', '2022-2-22', '2022-2-23', '2022-2-24', '2022-2-25', '2022-2-26', '2022-2-27', '2022-2-28', '2022-3-1', '2022-3-2', '2022-3-3', '2022-3-4', '2022-3-5', '2022-3-6', '2022-3-7', '2022-3-8', '2022-3-9', '2022-3-10', '2022-3-11', '2022-3-12', '2022-3-13', '2022-3-14', '2022-3-15', '2022-3-16', '2022-3-17', '2022-3-18', '2022-3-19', '2022-3-20', '2022-3-21', '2022-3-22', '2022-3-23', '2022-3-24', '2022-3-25', '2022-3-26', '2022-3-27', '2022-3-28'])

In [ ]:
ticker_dict

In [29]:
# comment_data = pd.read_csv('2022-02_2022-04_comment_data.csv')
comment_data = pd.DataFrame.from_dict(ticker_dict)
comment_data.to_csv('2022-02_2022-03_comment_data.csv')

In [30]:
comment_data

,GME,Gamestop,SPY,TWTTR,Twitter,TSLA,Tesla,AMD
2022-1-31,"[AMC GME 💎🤲🦍, I wish people had actual DD on G...",[],[Why wait? Just buy SPY puts now. Then calls w...,[],[],[TSLA Weekly 1000C for February- make that dam...,[I don’t see Tesla calls there mate your dd mi...,"[Nice Info, still going to YOLO in AMD C and M..."
2022-2-1,"[Yes yes, this is good and all, but what can y...",[Gamestop managed to turn 2 🌈🐻 back to straigh...,[Same guy who said SPY 5000 by January 31st an...,[],[],[],[Nio finishes car prototype: Onlookers call it...,[]
2022-2-2,[Bought an AMD CPU and Motherboard this past m...,[],[Just delete the app. I’m sure the problem wi...,[],[],[Everyone knows the infinite money glitch is T...,"[Microsoft, Facebook, Apple, Google, something...","[Bro you are a goat, investing +1m in Intel li..."
2022-2-3,[],[],[Get rich quick scheme!!!!!! Start IRA and con...,[],[I think FB and all the other woke social medi...,[],[Your going to be a machine when you get older...,"[There shouldn’t be any too big to fail stuff,..."
2022-2-4,[People like you make me embarrassed to hold m...,[],[They're up 4X in 10 years and SPY is up 3.5x....,[],[],[I hate to say it but I respectfully disagree....,[],[Agreed. When I started here guys were highlig...
2022-2-5,"[Unless you can show evidence to the contrary,...",[],[That Thursday pre-market conference call soun...,[],"[Unless you can show evidence to the contrary,...",[Too bad it’s a shit car.\n\nBuy TSLA puts],"[Unless you can show evidence to the contrary,...",[He also said AMD…]
2022-2-6,[GME apes literally live rent free in this guy...,[Just like selling Gamestop a game you paid $6...,[65% people are long on the market.. Last Mond...,[],[Putting every single cent I have into Twitter...,[TSLA $2000 EOY],[Musk announcement: Tesla will sell affordable...,[]
2022-2-9,"[Also we need to add AMC and GME, Thats cause ...","[Gamestop is the key..., Google, Apple, Nvidia...",[I stopped using Facebook recently. Cuz someon...,[],[],"[Elon, please split TSLA 20-1 so that TSLA can...",[Netflix is fired from FAANG. Tesla is gonna j...,[Since all you retards think this a therapy se...
2022-2-10,"[DRS them GME shares, Poggers, DRS GME ......a...",[],[7.5% and SPY most likely $480 next week.],[],[],[],[So buy Tesla orders now and profit??],[]
2022-2-11,[And $GME up 20% this week.],[],[> I plan on buying SPY puts just before the F...,[],[],[You say that but I've never mentally recovere...,"[“Tesla Gang” lmao 🤣, So you're saying Tesla s...",[]


## Making the Comments Data Frame

In [ ]:
analyzer = SentimentIntensityAnalyzer()

def get_list_of_scores(listofcomments):
    templist=[]
    
    for com in listofcomments:
        temp = analyzer.polarity_scores(com)
        compScore = temp['compound']
        templist.append(compScore)
            
    return templist

#need language if list greater 0
def get_means_comp(listofScores):
    return statistics.mean(listofScores)

#need language if list greater than 1
def returnspreadComp(listofScores):
    return statistics.stdev(listofScores)

In [ ]:
index = comment_data["Unnamed: 0"]
gme_com_data = comment_data['GME']
gme_df = pd.DataFrame()

gme_df.index = index
gme_df['Scores'] = get_list_of_scores(comment_data['GME'])
# gme_df['Mean Comp'] = get_means_comp(comment_data['GME'])
# gme_df['Spread Comp'] = returnspreadComp(comment_data['GME'])

In [ ]:
get_means_comp(comment_data['GME'])

In [ ]:
gme_df